# 📊 Task 3:  Restaurant Recommendation


**Internship:** Cognifyz Technologies  
**Intern:** Ashok R  
**Task Objective:** Create a restaurant recommendation
 system based on user preferences.

# Import Libraries & Load Dataset

In [141]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer


In [143]:
df = pd.read_csv(r"C:\Users\ashok\Downloads\Cognifyz_intern\Dataset .csv")
df.head()

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [145]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

In [147]:
df.shape

(9551, 21)

 # Data Cleaning

In [150]:
df.isnull().values.any()

np.True_

In [152]:
df.isnull().values.sum()

np.int64(9)

In [154]:
df.columns[df.isnull().any()]

Index(['Cuisines'], dtype='object')

In [156]:
df = df[['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating', 'City']]

df.dropna(subset=['Cuisines'], inplace=True)
df = df[df['Aggregate rating'] > 0]          # drop unrated rows
df.reset_index(drop=True, inplace=True)

# Preprocess for Recommendation

In [159]:
df['tag'] = (
    df['Cuisines'].str.replace('[^a-zA-Z, ]', '', regex=True) + ' ' +   # cuisines words
    df['City'].str.replace('[^a-zA-Z ]', '', regex=True)   + ' ' +
    df['Price range'].astype(str)
)

# Vectorization using TF-IDF

In [162]:
vectorizer = TfidfVectorizer(stop_words='english')
tag_matrix = vectorizer.fit_transform(df['tag'])


# Cosine Similarity Calculation

In [165]:
similarity = cosine_similarity(tag_matrix)


# Recommendation Function

In [167]:
# 7️Recommendation function
def recommend(restaurant_name, df=df, similarity=similarity, top_n=5):
    # find index of chosen restaurant
    try:
        idx = df[df['Restaurant Name'].str.lower() == restaurant_name.lower()].index[0]
    except IndexError:
        print("Restaurant not found, check spelling.")
        return
    
     # similarity scores & sort
    sim_scores = list(enumerate(similarity[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1: top_n+1]

    print(f"\nTop {top_n} recommendations for **{restaurant_name}**:\n")
    for i, score in sim_scores:
        row = df.iloc[i]
        print(f"{row['Restaurant Name']}  |  Cuisines: {row['Cuisines']}  |  Rating: {row['Aggregate rating']}  |  Similarity: {score:.2f}")


# Test with Example

In [171]:
recommend("Domino's Pizza", top_n=5)


Top 5 recommendations for **Domino's Pizza**:

Naivedya  |  Cuisines: North Indian  |  Rating: 3.4  |  Similarity: 0.80
Bhoj Restaurant  |  Cuisines: North Indian  |  Rating: 3.7  |  Similarity: 0.80
Kareem's Kabab & Biryani  |  Cuisines: Mughlai  |  Rating: 3.4  |  Similarity: 0.78
Angeethi Restaurant  |  Cuisines: North Indian, Chinese  |  Rating: 3.3  |  Similarity: 0.77
Great Sagar Restaurant  |  Cuisines: North Indian, Mughlai  |  Rating: 3.3  |  Similarity: 0.73
